### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [72]:
pd.read_excel?

In [2]:
# Read xls file 
path = os.getcwd() + "\\Data\\Energy Indicators.xls"
#path = os.getcwd() + "/Energy Indicators.xls"

with pd.ExcelFile(path) as con:
    df1 = pd.read_excel(io = path, sheetname = 'Energy',skiprows = 18, 
                        header = None, index_col = None,
                       names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],
                       dtype = {'Country' : 'str', 
                                'Energy Supply' : np.float64,  # will work even is NaN are there, as NaN is float
                                'Energy Supply per Capita' : np.float64,
                                '% Renewable' : np.float64},
                       # converters = {'Energy Supply' : peta_to_giga, },
                        parse_cols = "C:F", 
                        na_values = ['', 'NA', None, '...'], 
                        verbose = True)

# Conversion of gigaJ to petaJ    
df1['Energy Suppply(GJ)'] = df1['Energy Supply'].map(lambda x : x*1000000) 
df1.drop('Energy Supply', axis= 1,inplace = True)
df1.dropna(axis = 0, how = 'all',inplace = True) # drop empty rows

# Function to make Corrections 
def dic_corrections(x):
    '''
    1. Applies corrections stored in a dictionary to a string 
    2. Remove parts of string, if any in parentheses, and any numbers in the string
    
    '''
    cor_dic = {"South Korea" : [["Republic", 'Korea'], ["Rep.", 'Korea']],
            "United States" : [["United", "States", "America"], ],
            "United Kingdom" : [["United", "Kingdom", "Britain"], ["Northern", "Ireland"]] ,
            "Hong Kong"  : [['China',"Hong" ,"Kong"],],
            "Iran" : [["Iran",],]}
    
    lt_words = x.replace(',', '').split(sep = ' ')
    #print(lt_words)
    for k,vals in cor_dic.items():
        #print(k, vals)
        for lt in vals:
            check = all([el in lt_words for el in lt])
            #print(check)
            if(check & (k!= 'South Korea')):
                return(k)
            elif(check & (k== 'South Korea') & (len(lt_words) < 4)):
                return(k)
    return(x)

def other_corrections(x):
    # partiton on '(' and extract 1st component of the resulting list
    temp = re.search('\(', x) 
    if(temp is not None):
        result = x[0: temp.start()]
    else:
        result = x
    #print(result)    
    m = re.search('[0-9]+', result)  
    if(m is not None):
        result_fin = result.replace(result[m.start(): m.end()], '')
    else:
        result_fin = result
        #print(result)
    return result_fin.strip()
    
def correct(x):
    temp = other_corrections(x)
    return(dic_corrections(temp))

Reading sheet Energy


In [3]:
df1['Country_new'] = df1['Country'].map(correct)

In [97]:
#df1.set_index('Country_new',inplace= True)

---

### Extracting Data from Web, Zipped folders 

#### Module - requests 

- requests (http://docs.python-requests.org/en/master/user/quickstart/) provides easy ways to access url 
(Unique resource locations) for http, ftp protocols, allowing for handling potential errors and content access methods
- http url access can be thought as using a request response framework

- response = requests.get : sends a get request,and stores the response  

Content of reponse can be examined using methods like  
- response.text - Access content as strings, using the default encoding  
- response.content - Access the contest as Binary Bytes  
- response.encoding - check the encoding being used  
- response.json() - can access content as json also 
- zp = zipfile.ZipFile(io.BytesIO(response.content)) - for zip directories 
- zp.namelist() shows zipped files

In [101]:
type(response.content) # bytes 
type(response.text) # str 
print(response.encoding) # returns None

None


#### Module io  (https://docs.python.org/3/library/io.html#id1)

Module io provides functionality for different types of I/O : 
- text : the basic unit expected from read/write is string  
- binary or Buffered I/O : bsic unit is bytes  
- raw or UnBuffered I/O

#### Module zipfile

---

In [51]:
# Get data from web 
url1 = 'http://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=csv'
url2 = 'https://www.scimagojr.com/countryrank.php?category=2102&out=xls'

- Download file to a loaction  
- Inspect if file is zipped, if it is extract file names 
- Read a specific csv file present in a zipped dir in pandas df

In [75]:
pd.read_csv?

In [85]:
# import urllib
import io
from zipfile import ZipFile
import requests

In [88]:
response = requests.get(url1)
zp = ZipFile(io.BytesIO(response.content))

In [94]:
#response.content

In [90]:
#zf = ZipFile(url1, 'r')
zp.namelist()

['Metadata_Indicator_API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10051799.csv',
 'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10051799.csv',
 'Metadata_Country_API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10051799.csv']

In [77]:
response = urllib.request.urlopen(url1)

In [68]:
response.g

<bound method HTTPResponse.info of <http.client.HTTPResponse object at 0x0000027FD13DDF98>>

In [73]:
zp = ZipFile(io.BytesIO(response.read()))
zp.namelist()
#io.BytesIO(response.read())

BadZipFile: File is not a zip file

In [37]:
response = requests.get(url1)  # sends a get request
file = os.getcwd() +"\\Data\\gdp_1.csv"
with open(file, 'wt') as con:
    con.write(response.content)

TypeError: write() argument must be str, not bytes

In [34]:
with open(file , 'rb') as con:
    df1 = pd.read_csv(con)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 7: invalid start byte

In [31]:
open?